# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())
filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 

for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print('Total number of lines in the CSV file: '.format(sum(1 for line in f)))

Total number of lines in the CSV file: 


# Part II. Generating Queries

## The <font color=red>event_datafile_new.csv</font> contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS "eventData"
    WITH REPLICATION = 
    {'class': 'SimpleStrategy', 'replication_factor': 1}"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace("eventData")
except Exception as e:
    print(e)

## Query 1
### Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### Creating Table

In [8]:
query = "CREATE TABLE IF NOT EXISTS songs_data "
query = query + "(artist text, song text, length decimal, sessionId int, itemInSession int, PRIMARY KEY(sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Populating Table

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO songs_data (artist, song, length, sessionId, itemInSession) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[-2], float(line[5]), int(line[-3]), int(line[3])))

#### Printing the results of the Query

In [10]:
query = "select * from songs_data WHERE sessionId = 338 and itemInSession  = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

temp = []
for row in rows:
    temp.append((row.artist, row.song, row.length))

In [11]:
songs_data = pd.DataFrame(temp)
songs_data.columns = ('artist', 'song', 'length')
songs_data

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.3073


## Query 2
### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### Creating Table

In [12]:
query = "CREATE TABLE IF NOT EXISTS user_session_data "
query = query + "(user_id int, artist text, song text, sessionId int, itemInSession int, firstname text, lastname text, \
                    PRIMARY KEY((user_id, sessionId),itemInSession, song, firstname, lastname))"
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Populating Table

In [13]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO user_session_data (user_id, artist, song, sessionId, itemInSession, firstname, lastname) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[-1]), line[0], line[-2], int(line[-3]), int(line[3]), line[1], line[4]))                    

#### Printing the results of the Query

In [14]:
query = "select * from user_session_data where user_id = 10 and sessionId = 182 "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

temp = []
for row in rows:
    temp.append((row.artist, row.song, row.firstname, row.lastname))

In [15]:
user_session_data = pd.DataFrame(temp)
user_session_data.columns = ('artist', 'song', 'firstname', 'lastname')
user_session_data

,artist,song,firstname,lastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


## Query 3
### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Creating Table

In [16]:
query = "CREATE TABLE IF NOT EXISTS streaming_data "
query = query + "(song text, firstName text, lastName text, PRIMARY KEY(song, firstName, lastName))"
try:
    session.execute(query)
except Exception as e:
    print(e)             

#### Populating Table

In [17]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO streaming_data (song, firstName, lastName) "
        query = query + "VALUES (%s, %s, %s)"
        session.execute(query, (line[-2], line[1], line[4]))       

In [18]:
query = "select * from streaming_data where song = 'All Hands Against His Own' "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

temp = []
for row in rows:
    temp.append((row.firstname, row.lastname))

In [19]:
streaming_data = pd.DataFrame(temp)
streaming_data.columns = ('firstname', 'lastname')
streaming_data

,firstname,lastname
0,Jacqueline,Lynch
1,Sara,Johnson
2,Tegan,Levine


### Drop the tables before closing out the sessions

In [20]:
query = "drop table app_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table songs_data"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table users_data"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

Error from server: code=2200 [Invalid query] message="unconfigured table app_history"
Error from server: code=2200 [Invalid query] message="unconfigured table users_data"


### Close the session and cluster connection¶

In [21]:
session.shutdown()
cluster.shutdown()